In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Sjögrens_Syndrome"
cohort = "GSE94510"

# Input paths
in_trait_dir = "../../input/GEO/Sjögrens_Syndrome"
in_cohort_dir = "../../input/GEO/Sjögrens_Syndrome/GSE94510"

# Output paths
out_data_file = "../../output/preprocess/Sjögrens_Syndrome/GSE94510.csv"
out_gene_data_file = "../../output/preprocess/Sjögrens_Syndrome/gene_data/GSE94510.csv"
out_clinical_data_file = "../../output/preprocess/Sjögrens_Syndrome/clinical_data/GSE94510.csv"
json_path = "../../output/preprocess/Sjögrens_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"CD4 T-cells from pSS patients and human healthy volunteers"
!Series_summary	"Multi-omics study was conducted to elucidate the crucial molecular mechanisms of primary Sjögren’s syndrome (SS) pathology. We generated multiple data set from well-defined patients with SS, which includes whole-blood transcriptomes, serum proteomes and peripheral immunophenotyping. Based on our newly generated data, we performed an extensive bioinformatic investigation. Our integrative analysis identified SS gene signatures (SGS) dysregulated in widespread omics layers, including epigenomes, mRNAs and proteins. SGS predominantly involved the interferon signature and ADAMs substrates. Besides, SGS was significantly overlapped with SS-causing genes indicated by a genome-wide association study and expression trait loci analyses. Combining the molecular signatures with immunophenotypic profiles revealed that cytotoxic CD8 ­T cells­ were associated with SGS. Further, we obser

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains transcriptome arrays data from CD4 T-cells
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Trait data is available in row 0 with 'disease' indicating HC (healthy control) or pSS (primary Sjögren's syndrome)
trait_row = 0

# No age information is available in the sample characteristics
age_row = None

# Gender information is available in row 3, but it shows only one value 'Female', making it not useful for association studies
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait values to binary format (0 for healthy control, 1 for pSS)"""
    if not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert values
    if value.lower() == 'hc':
        return 0  # Healthy control
    elif value.lower() == 'pss':
        return 1  # Primary Sjögren's syndrome
    else:
        return None

def convert_age(value):
    """Placeholder function for age conversion"""
    return None

def convert_gender(value):
    """Placeholder function for gender conversion"""
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save cohort information for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    preview = preview_df(clinical_features)
    print("Clinical features preview:")
    print(preview)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical features preview:
{'GSM2477208': [0.0], 'GSM2477209': [0.0], 'GSM2477210': [0.0], 'GSM2477211': [0.0], 'GSM2477212': [0.0], 'GSM2477213': [0.0], 'GSM2477214': [0.0], 'GSM2477215': [0.0], 'GSM2477216': [0.0], 'GSM2477217': [0.0], 'GSM2477218': [0.0], 'GSM2477219': [0.0], 'GSM2477220': [0.0], 'GSM2477221': [0.0], 'GSM2477222': [0.0], 'GSM2477223': [0.0], 'GSM2477224': [0.0], 'GSM2477225': [0.0], 'GSM2477226': [1.0], 'GSM2477227': [1.0], 'GSM2477228': [1.0], 'GSM2477229': [1.0], 'GSM2477230': [1.0], 'GSM2477231': [1.0], 'GSM2477232': [1.0], 'GSM2477233': [1.0], 'GSM2477234': [1.0], 'GSM2477235': [1.0], 'GSM2477236': [1.0], 'GSM2477237': [1.0], 'GSM2477238': [1.0], 'GSM2477239': [1.0], 'GSM2477240': [1.0], 'GSM2477241': [1.0], 'GSM2477242': [1.0], 'GSM2477243': [1.0]}
Clinical data saved to ../../output/preprocess/Sjögrens_Syndrome/clinical_data/GSE94510.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers like '1007_s_at', '1053_at', etc. are Affymetrix probe IDs
# from microarray platforms, not standard human gene symbols.
# They need to be mapped to official gene symbols for analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. After observing the gene annotation dataframe, I can see that:
# - The 'ID' column contains probe IDs like '1007_s_at', which match the gene expression data index
# - The 'Gene Symbol' column contains the gene symbols like 'DDR1 /// MIR4640', which we need to map to

# 2. Get gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# This handles many-to-many relationships by dividing probe values equally among mapped genes
# and then summing all contributions for each gene
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the first few rows of the gene expression data after mapping
print("First 5 rows of gene expression data after mapping:")
print(gene_data.head())


First 5 rows of gene expression data after mapping:
          GSM2477208  GSM2477209  GSM2477210  GSM2477211  GSM2477212  \
Gene                                                                   
A1BG        5.889367    5.831167    6.226607    6.712019    5.585187   
A1BG-AS1    7.046769    6.399982    6.532333    6.985073    5.751564   
A1CF        7.063561    7.138271    8.557570    7.329160    7.397388   
A2M        13.870171   16.120248   17.433439   13.844135   15.039230   
A2M-AS1     7.309656    8.112662    8.443369    3.936552    7.390485   

          GSM2477213  GSM2477214  GSM2477215  GSM2477216  GSM2477217  ...  \
Gene                                                                  ...   
A1BG        4.984236    6.325255    5.547503    6.571947    6.099702  ...   
A1BG-AS1    6.145012    6.659559    6.293370    7.302131    6.706856  ...   
A1CF        8.897464    7.989230    7.009666    7.061376    7.266169  ...   
A2M        17.496613   11.361369   15.187867   14.157998  

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the previously saved clinical data
clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
print(f"Loaded clinical data shape: {clinical_df.shape}")
print(clinical_df.head())

# 3. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print(linked_data.head())

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains gene expression data from CD4 T-cells of pSS patients and healthy controls."
)

# 7. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")

Normalized gene data shape: (19845, 36)
First few normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']


Normalized gene data saved to ../../output/preprocess/Sjögrens_Syndrome/gene_data/GSE94510.csv
Loaded clinical data shape: (1, 36)
                   GSM2477208  GSM2477209  GSM2477210  GSM2477211  GSM2477212  \
Sjögrens_Syndrome         0.0         0.0         0.0         0.0         0.0   

                   GSM2477213  GSM2477214  GSM2477215  GSM2477216  GSM2477217  \
Sjögrens_Syndrome         0.0         0.0         0.0         0.0         0.0   

                   ...  GSM2477234  GSM2477235  GSM2477236  GSM2477237  \
Sjögrens_Syndrome  ...         1.0         1.0         1.0         1.0   

                   GSM2477238  GSM2477239  GSM2477240  GSM2477241  GSM2477242  \
Sjögrens_Syndrome         1.0         1.0         1.0         1.0         1.0   

                   GSM2477243  
Sjögrens_Syndrome         1.0  

[1 rows x 36 columns]
Linked data shape: (36, 19846)
            Sjögrens_Syndrome      A1BG  A1BG-AS1      A1CF        A2M  \
GSM2477208                0.0  5.889367

Shape after handling missing values: (36, 19846)
For the feature 'Sjögrens_Syndrome', the least common label is '0.0' with 18 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Sjögrens_Syndrome' in this dataset is fine.



Linked data saved to ../../output/preprocess/Sjögrens_Syndrome/GSE94510.csv
